To celebrate Thanksgiving, you and 19 of your family members are seated at a circular table (socially distanced, of course). Everyone at the table would like a helping of cranberry sauce, which happens to be in front of you at the moment.

Instead of passing the sauce around in a circle, you pass it randomly to the person seated directly to your left or to your right. They then do the same, passing it randomly either to the person to their left or right. This continues until everyone has, at some point, received the cranberry sauce.

Of the 20 people in the circle, who has the greatest chance of being the last to receive the cranberry sauce?

In [1]:
from collections import namedtuple
import numpy as np
import pandas as pd

# Numerical approach

It becomes clear pretty quickly from simulations that all positions round the table have an equal chance of being the last to receive the sauce (except position 0 where we are sitting and assuming that we have "received" the sauce initially i.e. `greedy=True` in the code below).

In [2]:
def run_simulation(N, greedy=True):
    current_position = 0
    set_positions_received = {0} if greedy else set()

    while len(set_positions_received) < N:
        move = 2 * (np.random.rand() < 0.5) - 1
        current_position = (current_position + move) % N
        set_positions_received.add(current_position)
        
    return current_position

In [3]:
pd.Series([run_simulation(20, greedy=True) for _ in range(100000)]).value_counts()

14    5409
15    5402
19    5378
9     5339
1     5323
16    5308
7     5308
18    5299
6     5289
8     5244
10    5241
5     5231
12    5228
2     5216
4     5188
17    5176
11    5164
3     5164
13    5093
dtype: int64

If we assume that we don't serve ourselves first (`greedy=False`), all positions including our own but excepting our immediate neighbours' have an equal chance of being last. They have half the chance of everyone else.

In [4]:
pd.Series([run_simulation(20, greedy=False) for _ in range(100000)]).value_counts()

15    5439
17    5378
0     5326
8     5322
5     5316
7     5313
10    5295
3     5274
14    5244
18    5222
16    5219
12    5214
4     5213
9     5202
6     5168
13    5166
11    5157
2     5154
1     2696
19    2682
dtype: int64

# Analytical approach

This combinatorial approach calculates the number of ways to visit each position last, as a proportion of the total (infinite) number of possible paths. It recognises that each position can be visited last from either a clockwise or anticlockwise direction. The two possible immediately previous positions can be viewed as two ends (bounds) of a number line along which we are conducting a random walk. The ways to exit this random walk can be expressed as an infinite series the proportional shares for each path length.

In [5]:
PathState = namedtuple('PathState', field_names=['all_paths', 'cw_complete_paths', 'acw_complete_paths'])

def pathfinder(N, last_position, limit=10):
    paths = {0: {0: PathState(1, 1 if last_position == N - 1 else 0, 1 if last_position == 1 else 0)}}
    acw_bound = last_position
    cw_bound = last_position - N
    acw_exits = []
    cw_exits = []
    for i in range(1, limit + 1):
        paths[i] = {}
        min_pos = max(cw_bound, min(paths[i-1].keys())-1)
        max_pos = min(acw_bound, max(paths[i-1].keys())+1)
        for k in range(min_pos, max_pos+1, 2):
            if k == cw_bound:
                cw_exits.append((paths[i-1][k+1].acw_complete_paths, f'2**{i}'))
            elif k == acw_bound:
                acw_exits.append((paths[i-1][k-1].cw_complete_paths, f'2**{i}'))
            else:
                all_paths = (getattr(paths[i-1].get(k-1, None), 'all_paths', 0)
                             + getattr(paths[i-1].get(k+1, None), 'all_paths', 0))
                cw_complete_paths = all_paths if k == (cw_bound + 1) else (
                    getattr(paths[i-1].get(k-1, None), 'cw_complete_paths', 0)
                    + getattr(paths[i-1].get(k+1, None), 'cw_complete_paths', 0)
                )
                acw_complete_paths = all_paths if k == (acw_bound - 1) else (
                    getattr(paths[i-1].get(k-1, None), 'acw_complete_paths', 0)
                    + getattr(paths[i-1].get(k+1, None), 'acw_complete_paths', 0)
                )
                paths[i][k] = PathState(all_paths, cw_complete_paths, acw_complete_paths)
    return paths, acw_exits, cw_exits

In [6]:
paths, acw_exits, cw_exits = pathfinder(5,1,18)
cw = [t for t in cw_exits if t[0] > 0]
acw = [t for t in acw_exits if t[0] > 0]
print(cw)
print(acw)
print(sum([t[0] / eval(t[1]) for t in cw + acw]))

[(1, '2**4'), (3, '2**6'), (8, '2**8'), (21, '2**10'), (55, '2**12'), (144, '2**14'), (377, '2**16'), (987, '2**18')]
[(1, '2**7'), (5, '2**9'), (18, '2**11'), (57, '2**13'), (169, '2**15'), (482, '2**17')]
0.23502731323242188
